In [ ]:
import cv2
import numpy as np
import random
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from tqdm import tqdm
import numpy as np
from PIL import Image
import os
import matplotlib.pyplot as plt
import random
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from tqdm import tqdm
import numpy as np
from PIL import Image
import os
from torch.optim import lr_scheduler
import imageio
from database2 import DehazingDataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Function to add haze in varying intensity

In [ ]:

def add_haze(image, haze_intensity = "low"):
    # Convert image to uint8 data type
    image_uint8 = (image * 255).astype(np.uint8)

    # Simulate haze by blending the image with a white overlay
    overlay = np.full_like(image_uint8, (255, 255, 255), dtype=np.uint8)  # White overlay

    # Define range of alpha values based on haze intensity
    if haze_intensity == 'low':
        # Adjusted alpha range for low haze to make it a little more hazy
        alpha_range = (0.2, 0.5)
    elif haze_intensity == 'medium':
        alpha_range = (0.3, 0.6)
    elif haze_intensity == 'high':
        alpha_range = (0.6, 0.9)
    else:
        raise ValueError("Invalid haze intensity level. Choose from 'low', 'medium', or 'high'.")

    # Random transparency level within the specified range
    alpha = random.uniform(alpha_range[0], alpha_range[1])

    # Blend image with overlay to create haze effect
    haze_image = cv2.addWeighted(image_uint8, 1 - alpha, overlay, alpha, 0)

    return haze_image


# Function to show clean image along with hazy images

In [ ]:
def show_images(dataloader, num_images=5, save_path = None, save_images = False, save_fig = False):
    # Get a batch of data
    data_iter = iter(dataloader)
    _, images = next(data_iter)

    # Plot original clean images
    fig, axes = plt.subplots(4, num_images, figsize=(15, 15))
    for i in range(num_images):
        clean_image = images[i].permute(1, 2, 0).cpu().numpy()  # Convert to NumPy array
        # clean_image = clean_image * 0.5 + 0.5  # Denormalize
        clean_image = clean_image * 0.5 + 0.5  # Denormalize and convert to uint8


        axes[0, i].imshow(clean_image)
        axes[0, i].axis('off')
        axes[0, i].set_title("Clean Image")

        # Add haze at different intensity levels
        hazy_image_low = add_haze(clean_image, 'low')
        axes[1, i].imshow(hazy_image_low)
        axes[1, i].axis('off')
        axes[1, i].set_title("Low Haze")

        hazy_image_medium = add_haze(clean_image, 'medium')
        axes[2, i].imshow(hazy_image_medium)
        axes[2, i].axis('off')
        axes[2, i].set_title("Medium Haze")

        hazy_image_high = add_haze(clean_image, 'high')
        axes[3, i].imshow(hazy_image_high)
        axes[3, i].axis('off')
        axes[3, i].set_title("High Haze")


    if save_path and save_images:
      # Image.fromarray(clean_image).save('clean_image_{i}.png')
      Image.fromarray(hazy_image_low).save('hazy_image_low_{i}.png')
      Image.fromarray(hazy_image_medium).save('hazy_image_medium_{i}.png')
      Image.fromarray(hazy_image_high).save('hazy_image_high_{i}.png')

    plt.tight_layout()

    if save_path and save_fig:
        plt.savefig(save_path)

    plt.show()

# Load Images into dataloader

In [ ]:
# root_dir = '/content/drive/MyDrive/Task2/Dataset'
root_dir = "Task2Dataset"
train_dir = os.path.join(root_dir, 'train')
val_dir = os.path.join(root_dir, 'val')
transform = transforms.Compose([
                                 transforms.ToTensor(),
                                 transforms.Normalize([0.5, 0.5, 0.5],[0.5, 0.5, 0.5])
                                 ])

train_dataset = DehazingDataset(train_dir, transform)
val_dataset = DehazingDataset(val_dir, transform)
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2)
val_dataloader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=2)

In [ ]:
def view_image(image):
  image = image.permute(1, 2, 0).cpu().numpy()  # Convert to NumPy array
        # clean_image = clean_image * 0.5 + 0.5  # Denormalize
  image = image * 0.5 + 0.5  # Denormalize and convert to uint8

  image = add_haze(image)
  Image.fromarray(image).save(f'clean_image_try.png')

  plt.imshow(image)



# See images & don't save

In [ ]:
show_images(train_dataloader)

# See images and save

In [ ]:
show_images(train_dataloader, save_path = 'VARYING_HAZE_AUGMENT.png', save_fig = True)

In [ ]:
show_images(train_dataloader, save_path = '/content/drive/MyDrive/hazy_images_augment_3.jpg')

# Actual Augmentation
## Read images from a directory, augment and save to respective directories

In [ ]:
def augment_and_save_images(dataloader, low_haze_dir, medium_haze_dir):
    # Create directories if they don't exist
    os.makedirs(low_haze_dir, exist_ok=True)
    os.makedirs(medium_haze_dir, exist_ok=True)

    count = 0
    for _, clean_imgs in dataloader:
        for i in range(len(clean_imgs)):
            clean_image = clean_imgs[i].permute(1, 2, 0).cpu().numpy()  # Convert to NumPy array
            clean_image = clean_image * 0.5 + 0.5  # Denormalize

            # Add haze at different intensity levels
            hazy_image_low = add_haze(clean_image, 'low')
            hazy_image_medium = add_haze(clean_image, 'medium')

            # Save only low haze and medium haze images
            Image.fromarray(hazy_image_low).save(os.path.join(low_haze_dir, f'hazy_image_low_{count}.png'))
            Image.fromarray(hazy_image_medium).save(os.path.join(medium_haze_dir, f'hazy_image_medium_{count}.png'))

            count += 1

            print(f"AUgmented {count} Image(s)")


In [ ]:
input_dir = '/content/drive/MyDrive/Task2/Dataset/train/GT'
low_haze_dir = '/content/drive/MyDrive/Task2/Dataset/augmented/low_haze'
medium_haze_dir = '/content/drive/MyDrive/Task2/Dataset/augmented/medium_haze'


In [ ]:
augment_and_save_images(train_dataloader, low_haze_dir, medium_haze_dir)

In [ ]:
clean_image

In [ ]:
image_files = [f for f in os.listdir(input_dir) if f.lower().endswith(('.jpg', '.png', '.jpeg'))]

In [ ]:
filename = "1.png"
image_path = os.path.join(input_dir, filename)
clean_image = cv2.imread(image_path)
clean_image = cv2.cvtColor(clean_image, cv2.COLOR_BGR2RGB)

In [ ]:
clean_image = Image.open(image_path)
clean_image = clean_image.convert("RGB")

In [ ]:
plt.imshow(clean_image)

In [ ]:
hazy_image_low = add_haze(clean_image, 'low')
plt.imshow(hazy_image_low)

In [ ]:
hazy_image_low.shape

In [ ]:
clean_image.shape

In [ ]:
hazy_image_low = cv2.cvtColor(hazy_image_low, cv2.COLOR_RGB2BGR)